<a href="https://colab.research.google.com/github/zendhiyb/Sentimen-Publik-Kasus-BEM-FISIP-UNAIR-Metode-Logistic-Regression-dan-Decision-Tree/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install slang

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.1/197.1 kB 13.6 MB/s eta 0:00:00
  Created wheel for slang: filename=slang-0.1.12-py3-none-any.whl size=59265 sha256=77febc6b8eb03c1850e78cdb7ea50c74661d435a52ef2d6950cff552b0d19f49
  Stored in directory: /root/.cache/pip/wheels/04/df/6c/af17149e2e0ad61edc770fdf56226e29f6db011567a57c4337
  Created wheel for atypes: filename=atypes-0.1.10-py3-none-any.whl size=15546 sha256=4d2bfaae21cafd83be38466f96ac1f5fb80d93292c4ba381ab243a996dc910c2
  Stored in directory: /root/.cache/pip/wheels/b1/61/f6/8886de36865422603126717b8d7af412c079e879c60da99724
Successfully built slang atypes


In [ ]:
!pip install indoNLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import json
from indoNLP.preprocessing import replace_slang
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.corpus import stopwords

# file CSV
file_path = '/content/drive/MyDrive/Tugas/MACHINE LEARNING/komentar_youtube_label.csv'  # Ganti dengan path file Anda yang sebenarnya
data = pd.read_csv(file_path)

# Cleaning
def clean_comment(comment):
    comment = re.sub(r'https?\S+', ' ', comment)
    comment = re.sub(r'@\S+', ' ', comment)
    comment = re.sub(r'#\S+', ' ', comment)
    comment = re.sub(r"\'\w+", ' ', comment)
    comment = re.sub(r'[^\w\s]', ' ', comment)
    comment = re.sub(r'\b2\b', ' ', comment)
    return comment.strip()

data['cleaned_comment'] = data['comment'].apply(clean_comment)
data = data[data['cleaned_comment'] != '']

# Case Folding
data['case_folded_comment'] = data['cleaned_comment'].str.lower()

# stopwords
nltk.download('stopwords')
stopwords_indonesia = stopwords.words('indonesian')
custom_stopwords = ['']
stopwords_indonesia.extend(custom_stopwords)

data['comment_without_stopwords'] = data['case_folded_comment'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stopwords_indonesia])
)

# slang
with open('/content/drive/MyDrive/Tugas/MACHINE LEARNING/slang.json', 'r', encoding='utf-8') as file:
    slang_dict = json.load(file)

def replace_slang_words(comment):
    words = comment.split()
    replaced_words = [slang_dict.get(word, word) for word in words]
    return ' '.join(replaced_words)

data['slang_replaced_comment'] = data['comment_without_stopwords'].apply(replace_slang_words)

# Stemming Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()
data['stemmed_comment'] = data['slang_replaced_comment'].apply(
    lambda x: ' '.join([stemmer.stem(word) for word in x.split()])
)

# Tokenisasi
data['tokens_final'] = data['stemmed_comment'].apply(lambda x: x.split())

output = data[['comment', 'cleaned_comment', 'case_folded_comment', 'comment_without_stopwords', 'slang_replaced_comment', 'stemmed_comment', 'tokens_final']].head(15)
output


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,comment,cleaned_comment,case_folded_comment,comment_without_stopwords,slang_replaced_comment,stemmed_comment,tokens_final
0,Ternyata ketua bem.nya cewek.barangkali ada yg...,Ternyata ketua bem nya cewek barangkali ada yg...,ternyata ketua bem nya cewek barangkali ada yg...,ketua bem nya cewek barangkali yg menunggangi ...,ketua bem nya cewek barangkali yang menunggang...,ketua bem nya cewek barangkali yang tunggang e...,"[ketua, bem, nya, cewek, barangkali, yang, tun..."
1,Ga usah ngomong etika bos la di ga punya etika...,Ga usah ngomong etika bos la di ga punya etika...,ga usah ngomong etika bos la di ga punya etika...,ga ngomong etika bos la ga etika bos,ga bicara etika bos lah ga etika bos,ga bicara etika bos lah ga etika bos,"[ga, bicara, etika, bos, lah, ga, etika, bos]"
2,Setuju bekukan BEM...tangkap antek2 asing...,Setuju bekukan BEM tangkap antek2 asing,setuju bekukan bem tangkap antek2 asing,setuju bekukan bem tangkap antek2 asing,setuju bekukan bem tangkap antek2 asing,tuju beku bem tangkap antek2 asing,"[tuju, beku, bem, tangkap, antek2, asing]"
3,Memag salah ngajar maha siswa jadi bodóh jàng...,Memag salah ngajar maha siswa jadi bodóh jàng...,memag salah ngajar maha siswa jadi bodóh jàng...,memag salah ngajar maha siswa bodóh jàngàn ßok...,memag salah ngajar maha siswa bodóh jàngàn ßok...,mag salah ngajar maha siswa bod h j ng n ok pi...,"[mag, salah, ngajar, maha, siswa, bod, h, j, n..."
4,akibat racun yg diberi rocky Gerung sudah ada ...,akibat racun yg diberi rocky Gerung sudah ada ...,akibat racun yg diberi rocky gerung sudah ada ...,akibat racun yg rocky gerung akibat ny anak ka...,akibat racun yang rocky gerung akibat ny anak ...,akibat racun yang rocky gerung akibat ny anak ...,"[akibat, racun, yang, rocky, gerung, akibat, n..."
5,Bekukan ajah Dia yg bikin Dia yg tdk tanggung ...,Bekukan ajah Dia yg bikin Dia yg tdk tanggung ...,bekukan ajah dia yg bikin dia yg tdk tanggung ...,bekukan ajah yg bikin yg tdk tanggung krn kara...,bekukan ajah yang bikin yang tidak tanggung ka...,beku ajah yang bikin yang tidak tanggung karen...,"[beku, ajah, yang, bikin, yang, tidak, tanggun..."
6,"Tunjukan kinerja data dan hasil yg baik,,,past...",Tunjukan kinerja data dan hasil yg baik past...,tunjukan kinerja data dan hasil yg baik past...,tunjukan kinerja data hasil yg hilang cacian n...,tunjukan kinerja data hasil yang hilang cacian...,tunjuk kerja data hasil yang hilang caci dan maki,"[tunjuk, kerja, data, hasil, yang, hilang, cac..."
7,Maju terus BEM,Maju terus BEM,maju terus bem,maju bem,maju bem,maju bem,"[maju, bem]"
8,"Tapi aku setuju di bekukan, buat suatu pelajar...",Tapi aku setuju di bekukan buat suatu pelajar...,tapi aku setuju di bekukan buat suatu pelajar...,setuju bekukan pelajaran biar ngga semena mena,setuju bekukan pelajaran biar tidak semena mena,tuju beku ajar biar tidak semena mena,"[tuju, beku, ajar, biar, tidak, semena, mena]"
9,Ini penghinaan bukan kritik maka harus dibekukan,Ini penghinaan bukan kritik maka harus dibekukan,ini penghinaan bukan kritik maka harus dibekukan,penghinaan kritik dibekukan,penghinaan kritik dibekukan,hina kritik beku,"[hina, kritik, beku]"


In [ ]:
data['text_string'] = data['tokens_final'].apply(lambda x: ' '.join([item for item in x if len(item)>3]))
data.head(5)

,comment,channel_name,date,like_count,dislike_count,sentiment,cleaned_comment,case_folded_comment,comment_without_stopwords,slang_replaced_comment,stemmed_comment,tokens_final,text_string
0,Ternyata ketua bem.nya cewek.barangkali ada yg...,@NurHidayat-ue7qs,2024-10-29 06:47:20+00:00,0,0,0.0,Ternyata ketua bem nya cewek barangkali ada yg...,ternyata ketua bem nya cewek barangkali ada yg...,ketua bem nya cewek barangkali yg menunggangi ...,ketua bem nya cewek barangkali yang menunggang...,ketua bem nya cewek barangkali yang tunggang e...,"[ketua, bem, nya, cewek, barangkali, yang, tun...",ketua cewek barangkali yang tunggang enak yang...
1,Ga usah ngomong etika bos la di ga punya etika...,@SulkanFaisal,2024-10-29 06:47:16+00:00,0,0,0.0,Ga usah ngomong etika bos la di ga punya etika...,ga usah ngomong etika bos la di ga punya etika...,ga ngomong etika bos la ga etika bos,ga bicara etika bos lah ga etika bos,ga bicara etika bos lah ga etika bos,"[ga, bicara, etika, bos, lah, ga, etika, bos]",bicara etika etika
2,Setuju bekukan BEM...tangkap antek2 asing...,@nandiasgar7017,2024-10-29 06:44:23+00:00,0,0,0.0,Setuju bekukan BEM tangkap antek2 asing,setuju bekukan bem tangkap antek2 asing,setuju bekukan bem tangkap antek2 asing,setuju bekukan bem tangkap antek2 asing,tuju beku bem tangkap antek2 asing,"[tuju, beku, bem, tangkap, antek2, asing]",tuju beku tangkap antek2 asing
3,Memag salah ngajar maha siswa jadi bodóh jàng...,@martingah689,2024-10-29 06:43:24+00:00,0,0,0.0,Memag salah ngajar maha siswa jadi bodóh jàng...,memag salah ngajar maha siswa jadi bodóh jàng...,memag salah ngajar maha siswa bodóh jàngàn ßok...,memag salah ngajar maha siswa bodóh jàngàn ßok...,mag salah ngajar maha siswa bod h j ng n ok pi...,"[mag, salah, ngajar, maha, siswa, bod, h, j, n...",salah ngajar maha siswa pinter
4,akibat racun yg diberi rocky Gerung sudah ada ...,@RuminiRumini-j1p,2024-10-29 06:33:52+00:00,1,0,0.0,akibat racun yg diberi rocky Gerung sudah ada ...,akibat racun yg diberi rocky gerung sudah ada ...,akibat racun yg rocky gerung akibat ny anak ka...,akibat racun yang rocky gerung akibat ny anak ...,akibat racun yang rocky gerung akibat ny anak ...,"[akibat, racun, yang, rocky, gerung, akibat, n...",akibat racun yang rocky gerung akibat anak kampua


In [ ]:
data.to_csv('komentar_youtube_preprocessing.csv', index=False)